In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

data_path \
    = 'https://raw.githubusercontent.com/fclesio/learning-space/master/Datasets/02%20-%20Classification/default_credit_card.csv'

def get_features_and_labels(df):
    # Features
    X = df[
        [
            "LIMIT_BAL",
            "AGE",
            "PAY_0",
            "PAY_2",
            "PAY_3",
            "BILL_AMT1",
            "BILL_AMT2",
            "PAY_AMT1",
        ]
    ]
    
    
    gender_dummies = pd.get_dummies(df[["SEX"]].astype(str))
    X = pd.concat([X, gender_dummies], axis=1)

    # Labels
    y = df["DEFAULT"]
    return X, y


def get_results(y_test, y_pred):
    acc = metrics.accuracy_score(y_test, y_pred)
    acc = round(acc, 2) * 100
    
    df_results = pd.DataFrame(y_pred)
    df_results.columns = ["status"]
    
    print(f"Accuracy: {acc}%")
    print(df_results.groupby(by=["status"]).size())

df = pd.read_csv(data_path)

X, y = get_features_and_labels(df)

X_train, X_test, y_train, y_test \
    = train_test_split(X, y, test_size=0.1, random_state=42)


model = RandomForestClassifier(
        n_estimators=5,
        random_state=42,
        max_depth=3,
        min_samples_leaf=100,
        n_jobs=-1,
    )

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

get_results(y_test, y_pred)

pickle.dump(model, open("model_rf.pkl", 'wb'))

Accuracy: 82.0%
status
0    2655
1     345
dtype: int64


In [2]:
!ls

ataque-cadeia-insumo.ipynb model_rf.pkl
model-training.py


In [3]:
import hashlib

def get_digest(file_path):
    '''Ref: https://stackoverflow.com/a/44873382/7024760'''
    h = hashlib.sha256()

    with open(file_path, 'rb') as file:
        while True:
            # Reading is buffered, so we can read smaller chunks.
            chunk = file.read(h.block_size)
            if not chunk:
                break
            h.update(chunk)

    return h.hexdigest()

In [4]:
get_digest('model_rf.pkl')

'fb6bef001d520e77a5835f933e27203c86e30ba789172e48425b3df83aa0108a'

### Ataque 

Nesse caso, temos um modelo pré-treinado que será transferido para outro local, por exemplo, de alguma equipe de Data Science para uma equipe de engenharia de Machine Learning.

O ataque consiste em pegar este modelo, fazer uma pequena modificação que pode ser prejudicial, e colocá-lo novamente no fluxo de ML, que neste caso é o pipeline de Produção do ML.

In [5]:
model_rf_reload_pkl \
    = pickle.load(open('model_rf.pkl', 'rb'))

model_rf_reload_pkl.classes_

array([0, 1])

In [6]:
# Ataque: Modificar todas as classes para 0
model_rf_reload_pkl.classes_ = np.array([0, 0])

In [7]:
model_rf_reload_pkl.classes_

array([0, 0])

In [8]:
pickle.dump(model_rf_reload_pkl, open("model_rf.pkl", 'wb'))

In [9]:
y_pred \
    = model_rf_reload_pkl.predict(X_test)

In [10]:
get_results(y_test, y_pred)

Accuracy: 78.0%
status
0    3000
dtype: int64


In [11]:
get_digest('model_rf.pkl')

'c81d8735169a8cb6460ec8b6cb55cc83f2dd6412a65181110df705ca21c4f1ab'

Como podemos ver, basta apenas o acesso ao objeto do Scikit-learn usando o Pickle para que todas as propriedades sejam acessíveis.


### Contramedidas
    - Se houver algum risco de algum tipo de intermediação nos pontos de contato dos modelos de ML (por exemplo, uma equipe DS, faz a transferência para uma equipe MLE e depois para outra equipe), é adequado usar hash SHA1 ou MD5 desde o início garantir a integridade do arquivo entre todas as entidades envolvidas com o modelo;
   
    - Se possível, tenha menos intermediários possível entre o treinamento do modelo e o _deployment_